In [1]:
!pip install transformers torchaudio soundfile

   ---------------------------------------- 0.0/2.4 MB ? eta -:--:--
   - -------------------------------------- 0.1/2.4 MB 1.7 MB/s eta 0:00:02
   ---- ----------------------------------- 0.3/2.4 MB 2.8 MB/s eta 0:00:01
   --------- ------------------------------ 0.6/2.4 MB 3.9 MB/s eta 0:00:01
   ----------------- ---------------------- 1.0/2.4 MB 5.5 MB/s eta 0:00:01
   ---------------------------- ----------- 1.7/2.4 MB 7.3 MB/s eta 0:00:01
   ---------------------------------------  2.4/2.4 MB 8.4 MB/s eta 0:00:01
   ---------------------------------------  2.4/2.4 MB 8.4 MB/s eta 0:00:01
   ---------------------------------------  2.4/2.4 MB 8.4 MB/s eta 0:00:01
   ---------------------------------------  2.4/2.4 MB 8.4 MB/s eta 0:00:01
   ---------------------------------------  2.4/2.4 MB 8.4 MB/s eta 0:00:01
   ---------------------------------------  2.4/2.4 MB 8.4 MB/s eta 0:00:01
   ---------------------------------------  2.4/2.4 MB 8.4 MB/s eta 0:00:01
   ----------------

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torchvision 0.16.0 requires torch==2.1.0, but you have torch 2.5.1 which is incompatible.


In [2]:
import torch
from transformers import Wav2Vec2ForCTC, Wav2Vec2Processor
import torchaudio
import urllib.request
import os

DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
MODEL_NAME = "facebook/wav2vec2-base-960h"

processor = Wav2Vec2Processor.from_pretrained(MODEL_NAME)
model = Wav2Vec2ForCTC.from_pretrained(MODEL_NAME).to(DEVICE).eval()

c:\Users\DELL\anaconda3\envs\ai-backend\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
tokenizer_config.json: 100%|██████████| 163/163 [00:00<00:00, 54.3kB/s]
config.json: 100%|██████████| 1.60k/1.60k [00:00<00:00, 319kB/s]
vocab.json: 100%|██████████| 291/291 [00:00<00:00, 63.3kB/s]
special_tokens_map.json: 100%|██████████| 85.0/85.0 [00:00<00:00, 16.7kB/s]
model.safetensors: 100%|██████████| 378M/378M [02:41<00:00, 2.34MB/s] 
Some weights of the model checkpoint at facebook/wav2vec2-base-960h were not used when initializing Wav2Vec2ForCTC: ['wav2vec2.encoder.pos_conv_embed.conv.weight_g', 'wav2vec2.encoder.pos_conv_embed.conv.weight_v']
- This IS expected if you are initializing Wav2Vec2ForCTC from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenc

In [6]:
# Specify your local audio file here.
# Make sure this file exists in the current directory.
sample_file = "C:/Users/DELL/Desktop/VOIP_Phishing_Attacks/Model Building/speech-text-conversion/assets/sample1.wav"


In [7]:
# Load the audio
speech, sr = torchaudio.load(sample_file)
# Resample if necessary (Wav2Vec2 expects 16kHz)
if sr != 16000:
    resampler = torchaudio.transforms.Resample(sr, 16000)
    speech = resampler(speech)
    sr = 16000

# Convert to a single channel if stereo
if speech.shape[0] > 1:
    speech = torch.mean(speech, dim=0, keepdim=True)

In [8]:
# Now run inference
input_values = processor(speech.squeeze().numpy(), return_tensors="pt", sampling_rate=sr).input_values.to(DEVICE)

with torch.no_grad():
    logits = model(input_values).logits
predicted_ids = torch.argmax(logits, dim=-1)
transcription = processor.batch_decode(predicted_ids)[0]
print("Transcription:", transcription)


Transcription: THIS IS A SOMPLE ODIO TO CHECK WITHER THE MODERRY DECTS THE SPEECH FROM PERLI


## Simulated Streaming Approach:
To mimic real-time processing, we’ll break the audio into smaller chunks and process them sequentially. This simulates receiving small audio segments continuously.

In [9]:
import math

chunk_duration = 1.0  # 1 second chunks
chunk_size = int(sr * chunk_duration)

audio_data = speech.squeeze().numpy()
total_samples = len(audio_data)
num_chunks = math.ceil(total_samples / chunk_size)

# We'll keep a rolling buffer to simulate partial transcripts
rolling_buffer = []
for i in range(num_chunks):
    start = i * chunk_size
    end = min((i+1)*chunk_size, total_samples)
    chunk = audio_data[start:end]

    input_values = processor(chunk, return_tensors="pt", sampling_rate=sr).input_values.to(DEVICE)

    with torch.no_grad():
        logits = model(input_values).logits
    predicted_ids = torch.argmax(logits, dim=-1)
    partial_transcription = processor.batch_decode(predicted_ids)[0]
    
    # Print the transcription of the current chunk
    print(f"Chunk {i+1}/{num_chunks} transcription: {partial_transcription}")


Chunk 1/7 transcription: THIS ISU
Chunk 2/7 transcription: COMPERT ODIO
Chunk 3/7 transcription: TO CHECK WI
Chunk 4/7 transcription: THA THE MOTER DE
Chunk 5/7 transcription: SPEECH
Chunk 6/7 transcription: FROM THE
Chunk 7/7 transcription: 
